<a href="https://colab.research.google.com/github/MinhongW/text_generation/blob/main/fine_tune_t5_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summarization - Text-to-Table Generation

## Background
- Documentation based on MDT is a widely in placed process across different value streams
- Documentation is a time-consuming process
- The current version of AutoDoc is rule-based and does not achieve automatic text generation

## Objectives
- Automatically generate descriptions for tables
- Build task specific and domain specific language model for text summarization
- The model will be on-prem without relying on OpenAI's API (so the data will not be passed to OpenAI)

In [ ]:
import pandas as pd
import json
import math
import platform
import sys
import tensorflow as tf

Summarize the task

What's T5 model

Summary of the experiments including the specs of data etc

List the challenges: data is small, t-small ,search for the hyper parameter

Error analysis

In [ ]:
! pip install datasets transformers rouge-score nltk sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
print(f"Python Platform: {platform.platform()}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: Linux-5.10.147+-x86_64-with-glibc2.31

Python 3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0]
Pandas 1.5.3
GPU is available


In [ ]:
!git clone https://github.com/MinhongW/text_generation.git

Cloning into 'text_generation'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 40 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (40/40), 1.59 MiB | 2.66 MiB/s, done.


In [ ]:
data_folder = 'text_generation/data'
t1 = open(f'{data_folder}/table_train.json')
t2 = open(f'{data_folder}/table_desc_train.json')
t3 = open(f'{data_folder}/paper_train.json')

v1 = open(f'{data_folder}/table_val.json')
v2 = open(f'{data_folder}/table_desc_val.json')
v3 = open(f'{data_folder}/paper_val.json')

te1 = open(f'{data_folder}/table_test.json')
te2 = open(f'{data_folder}/table_desc_test.json')
te3 = open(f'{data_folder}/paper_test.json')

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# t1 = open('table_train.json')
# t2 = open('table_desc_train.json')
# t3 = open('paper_train.json')

# v1 = open('table_val.json')
# v2 = open('table_desc_val.json')
# v3 = open('paper_val.json')

# te1 = open('table_test.json')
# te2 = open('table_desc_test.json')
# te3 = open('paper_test.json')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
tables_train = json.load(t1)
descs_train = json.load(t2)
papers_train = json.load(t3)

tables_val = json.load(v1)
descs_val = json.load(v2)
papers_val = json.load(v3)

tables_test = json.load(te1)
descs_test = json.load(te2)
papers_test = json.load(te3)

In [ ]:
# double check the order in table file and desc file

for i in range(len(tables_train)):
    table_id1 = tables_train[i]['table_id_paper']
    table_id2 = descs_train[i]['table_id_paper']
    if table_id1 != table_id2:
               print('oops')

# Naive representation

Simply flatten T into a sequence ignoring its table structure by concatenating captions, headers, metrics and targeted cell values.

In [ ]:
tables_train[0]

{'table_id_paper': 'D16-1007table_2',
 'caption': 'Comparison of different position features.',
 'row_header_level': 2,
 'row_headers': [['Position Feature', 'plain text PF'],
  ['Position Feature', 'TPF1'],
  ['Position Feature', 'TPF2']],
 'column_header_level': 1,
 'column_headers': [['F1']],
 'contents': [['83.21'], ['83.99'], ['83.90']],
 'metrics_loc': 'column',
 'metrics_type': ['F1'],
 'target_entity': ['Position Feature'],
 'table_html_clean': "<table border='1' class='dataframe'>  <thead>    <tr style='text-align: right;'>      <th></th>      <th>F1</th>    </tr>  </thead>  <tbody>    <tr>      <td>Position Feature || plain text PF</td>      <td>83.21</td>    </tr>    <tr>      <td>Position Feature || TPF1</td>      <td>83.99</td>    </tr>    <tr>      <td>Position Feature || TPF2</td>      <td>83.90</td>    </tr>  </tbody></table>",
 'table_name': 'Table 2',
 'table_id': 'table_2',
 'paper_id': 'D16-1007',
 'page_no': 8,
 'dir': 'emnlp2016',
 'valid': 1}

In [ ]:
descs_train[0]

{'table_id_paper': 'D16-1007table_2',
 'description': 'Table 2 summarizes the performances of proposed model when different position features are exploited. To concentrate on studying the effect of position features, we do not involve lexical features in this section. As the table shows, the position feature on plain text is still effective in our model and we accredit its satisfactory result to the dependency information and tree-based kernels. The F1 scores of tree-based position features are higher since they are “specially designed” for our model. Contrary to our expectation, the more fine-grained TPF2 does not yield a better performance than TPF1, and two kinds of TPF give fairly close results. One possible reason is that the influence of a more elaborated definition of relative position is minimal. As most sentences in this dataset are of short length and their dependency trees are not so complicated, replacing TPF1 with TPF2 usually brings little new structural information and t

In [ ]:
def naive_representation(tables, descs):
    """
    Input_text is generated by naive representation of the tables.
    Each table is simply flattened into a sequence ignoring its table structure
    by concatenating captions, headers, metrics and targeted cell values.
    Target_text is the description of the corresponding table.
    Returns a df contains input_text and target_text
    
    """
    
    data = {'input_text':[],
           'target_text':[]}
    
    for i in range(len(tables)):
        table = tables[i]
        caption = table['table_id'] + ' ' + table['caption']
        row_names = ' '.join(' '.join(x) for x in table['row_headers'])
        col_names = ' '.join(' '.join(x) for x in table['column_headers'])
        metrics = ' '.join(table['metrics_type'])
        values = ' '.join(' '.join(x) for x in table['contents'])        
        tmp = [caption, row_names, col_names, metrics, values]
        text = ' '.join(tmp)
        
        desc = descs[i]['description']        
        
        data['input_text'].append(text)
        data['target_text'].append(desc)
    
    df = pd.DataFrame(data)
    
    return df

In [ ]:
df_train = naive_representation(tables_train, descs_train)
df_val = naive_representation(tables_val, descs_val)
df_test = naive_representation(tables_test, descs_test)

In [ ]:
df_train.head()

,input_text,target_text
0,table_2 Comparison of different position featu...,Table 2 summarizes the performances of propose...
1,table_3 Pearson correlation values between hum...,Table 3 presents the correlation results for t...
2,table_4 Comparison between rationale models (m...,Results. Table 4 presents the results of our r...
3,table_2 Spearman’s rank correlation results on...,Table 2 shows the results of our contextdepend...
4,table_4 Examples of attention weights in diffe...,"From Table 4, we can find that in the first ho..."


In [ ]:
#df_train['target_text'][3]

In [ ]:
from datasets import Dataset
ds_train = Dataset.from_pandas(df_train)
ds_val = Dataset.from_pandas(df_val)
ds_test = Dataset.from_pandas(df_test)

In [ ]:
ds_train

Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 1084
})

In [ ]:
print(ds_val.shape[0])
print(ds_test.shape[0])

136
135


# T5 MODEL

In [ ]:
# import torch
# from transformers import T5Tokenizer, T5ForConditionalGeneration, Adafactor

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [ ]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


## Training with trainer API

In [ ]:
#from transformers import AutoTokenizer
from transformers import T5Tokenizer

#from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import T5ForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_metric

In [ ]:
model_checkpoint = "t5-small"

# AutoTokenizer will automatically select T5tokenizer as we specify a T5 model architecture here

tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer.model_max_length

512

In [ ]:
def tokenize_function(examples):
    model_inputs = tokenizer(examples["input_text"], padding="max_length", truncation=True)
    # Setup the tokenizer for targets
    # change as_target_tokenizer() to text_target later
    with tokenizer.as_target_tokenizer():
      labels = tokenizer(examples["target_text"], padding="max_length", truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

ds_train_tokenized = ds_train.map(tokenize_function, batched=True)
ds_val_tokenized = ds_val.map(tokenize_function, batched=True)
ds_test_tokenized = ds_test.map(tokenize_function, batched=True)

Map:   0%|          | 0/1084 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/136 [00:00<?, ? examples/s]

Map:   0%|          | 0/135 [00:00<?, ? examples/s]

In [ ]:
ds_train_tokenized

Dataset({
    features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1084
})

In [ ]:
#ds_train_tokenized['labels'][0]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:
batch_size = 4 # batch_size need to be tuned. set it as 4 here due to the memory limits
model_name = "t5-test"
model_dir = f"model/{model_name}"

In [ ]:
# random search
# AutoTuner from Transformer ==== check this

In [ ]:
# just for building the structure, change it later
args = Seq2SeqTrainingArguments(
    output_dir=model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1"
    #report_to="tensorboard",
)

In [ ]:
#!pip install evaluate

In [ ]:
import nltk
nltk.download('punkt')
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import numpy as np

metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them. (when model is unable to make a confident prediction)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence ######### need to double check here
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    # Return rouge1, rouge2, rougeL, rougeLsum
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# # Function that returns an untrained model to be trained
# def model_init():
#     # Freeze the first 2 layers of the encoder and the last 2 layers of the decoder
#     model_init = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
#     for name, param in model_init.named_parameters():
#         if 'encoder.block.0' in name or 'encoder.block.1' in name or 'decoder.block.10' in name or 'decoder.block.11' in name:
#             param.requires_grad = False
#     return model_init

# Function that returns an untrained model to be trained
def model_init():
    return T5ForConditionalGeneration.from_pretrained(model_checkpoint)

model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
# for name, param in model.named_parameters():
#     if 'encoder.block.0' in name or 'encoder.block.1' in name or 'decoder.block.10' in name or 'decoder.block.11' in name:
#         param.requires_grad = False


# use seq2seqtrainer (sub class of trainer)
# cause we need to predict with generate and evaluate with rouge
trainer = Seq2SeqTrainer(
    #model_init=model_init, # model_init has to be callable
    model=model,
    args=args,
    train_dataset=ds_train_tokenized,
    eval_dataset=ds_val_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
# compute_loss could be used to customise loss function
# CrossEntropyLoss is the default loss function used in seq2seq task using T5 model

In [ ]:
# seq2seqtrainer will automatically use gpu

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
100,5.131700,1.664425,0.000000,0.000000,0.000000,0.000000,0.000000
200,1.598900,1.498981,0.007000,0.000000,0.007000,0.007000,0.139700
300,1.620500,1.452515,0.791200,0.358900,0.642900,0.708500,1.117600
400,1.489100,1.429782,14.454400,5.828900,11.682300,13.633700,16.066200
500,1.441000,1.413815,17.049700,6.899300,14.040500,16.136100,18.441200
600,1.380100,1.403113,17.018500,7.020700,13.977200,16.144600,18.720600
700,1.445000,1.393210,17.656700,7.198400,14.344600,16.599800,19.000000
800,1.453900,1.384918,17.852800,7.205400,14.350300,16.799400,19.000000
900,1.396000,1.378428,17.599000,7.276700,14.155900,16.629400,18.941200
1000,1.395900,1.372403,17.848100,7.393300,14.251500,16.760500,19.000000


TrainOutput(global_step=2710, training_loss=1.5356905307277102, metrics={'train_runtime': 1707.93, 'train_samples_per_second': 6.347, 'train_steps_per_second': 1.587, 'total_flos': 1467105127956480.0, 'train_loss': 1.5356905307277102, 'epoch': 10.0})

In [ ]:
trainer.save_model()

In [ ]:
#model_dir = f"model/{model_name}"

run_name = "t5-small-v1.0"
model_folder = "text_generation/models"
model_dir = f"{model_folder}/{run_name}"
#tokenizer = AutoTokenizer.from_pretrained(model_dir)
trainer.save_model(model_dir)
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)
model = T5ForConditionalGeneration.from_pretrained(model_dir)

max_input_length = 512

In [ ]:
%cd text_generation

/content/text_generation


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
#!git init

Reinitialized existing Git repository in /content/text_generation/.git/


In [ ]:
df_test['input_text'][1]

'table_3 Comparison on validation perplexity. BoW, TDNN and NABS are the baseline neural compression models with different encoders in Rush et al. (2015) Model Bag-of-Word (BoW) Model Convolutional (TDNN) Model Attention-Based (NABS) (Rush et al.,,2015) Model Forced-Attention (FSC) Model Auto-encoding (ASC+FSC1) Labelled Data Perplexity Labelled Data Perplexity 3.8M 43.6 3.8M 35.9 3.8M 27.1 3.8M 18.6 3.8M 16.6'

In [ ]:
df_test['target_text'][1]

'In Figure 3, we present the validation perplexity to compare the abilities of the three models to learn the compression languages. The ASC+FSC1(red) employs the same dataset for unlabelled and labelled training, while the ASC+FSC2(black) employs the full unlabelled dataset. Here, the joint ASC+FSC1 model obtains better perplexities than the single discriminative FSC model, but there is not much difference between ASC+FSC1 and ASC+FSC2 when the size of the labelled dataset grows. From the perspective of language modelling, the generative ASC model indeed helps the discriminative model learn to generate good summary sentences. Table 3 displays the validation perplexities of the benchmark models, where the joint ASC+FSC1 model trained on the full labelled and unlabelled datasets performs the best on modelling compression languages.'

In [ ]:
input_text = df_test['input_text'][1]


input_ids = tokenizer.encode(input_text, truncation=True, return_tensors='pt')

# inputs = tokenizer(text, max_length=max_input_length, truncation=True, return_tensors="pt")
# output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
# decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
# predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]
output = model.generate(input_ids=input_ids, max_length=512)
tokenizer.decode(output[0])

'<pad> Table 3 shows the results of the model on validation perplexity. The neural compression model with different encoders is a baseline model with different encoders. The model is a baseline model with different encoders. The model is a baseline model with different encoders.</s>'

In [ ]:
ds_test_tokenized

Dataset({
    features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 135
})

## Evluate the model on the test set

In [ ]:
# import torch

# model.to(device)

# # prepare dataloader
# ds_test_tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask'])
# dataloader = torch.utils.data.DataLoader(ds_test_tokenized, batch_size=4)

# dataloader.to(device)

# # generate text for each batch
# all_predictions = []
# for i,batch in enumerate(dataloader):
#   predictions = model.generate(**batch, max_length=512)
#   all_predictions.append(predictions)

# # flatten predictions
# all_predictions_flattened = [pred for preds in all_predictions for pred in preds]

# # tokenize and pad titles
# all_target_texts = tokenizer(ds_test_tokenized["target_text"], max_length=512,
#                              truncation=True, padding="max_length")["input_ids"]

# # compute metrics
# predictions_labels = [all_predictions_flattened, all_target_texts]
# compute_metrics(predictions_labels)

In [ ]:
from torch.utils.data import DataLoader

model.to(device)

# define the collate function
def collate_fn(inputs):
    # tokenize the inputs and return a dictionary with input_ids and their lengths
    input_ids = tokenizer.batch_encode_plus(inputs, padding=True, truncation=True, return_tensors='pt')
    input_ids = input_ids['input_ids'].to(device)
    input_lengths = torch.sum(input_ids != tokenizer.pad_token_id, dim=1)
    return {'input_ids': input_ids, 'input_lengths': input_lengths}

# create the DataLoader
test_dataset = DataLoader(ds_test['input_text'], batch_size=4, collate_fn=collate_fn)

generated_outputs = []

model.eval()
with torch.no_grad():
    for batch in test_dataset:
        # move the batch to the GPU
        batch = {k: v.to(device) for k, v in batch.items()}
        # pass the inputs through the model to generate output
        generated_ids = model.generate(
            input_ids=batch['input_ids'],
            attention_mask=batch['input_ids'].ne(tokenizer.pad_token_id),
            max_length=512,
            num_beams=4,   # double check here
            early_stopping=True
        )
        # convert generated ids to text
        generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        # append generated texts to the output list
        generated_outputs.extend(generated_texts)

In [ ]:
#df_test.head()

In [ ]:
target_texts = df_test['target_text'].to_list() # a list of target strings

# Rouge expects a newline after each sentence
generated_outputs = ["\n".join(nltk.sent_tokenize(output.strip()))
                    for output in generated_outputs]
target_texts = ["\n".join(nltk.sent_tokenize(label.strip())) 
                for label in target_texts]


scores = metric.compute(predictions=generated_outputs, references=target_texts,
                    use_stemmer=True)
scores = {key: value.mid.fmeasure * 100 for key, value in scores.items()}
#scores = {key: round(value.mid.fmeasure, 4) * 100 for key, value in scores.items()}

In [ ]:
scores

In [ ]:
# # define the target texts
# target_texts = df_test['target_text'].to_list() # a list of target strings

# scores = metric.compute(predictions=generated_outputs, references=target_texts,
#                     use_stemmer=True) # double check use_stemmer
# # extract the F1 score
# #f1_score = scores[0]['rouge-1']['f']
# # add the F1 score to the list of Rouge scores
# #rouge_scores.append(f1_score)
# scores = {key: value.mid.fmeasure * 100 for key, value in scores.items()}

## Old code for fine-tuning T5 model

In [ ]:
# Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# need to double check here, do we need to use prefix for t5
model_checkpoint = 't5-small'
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small', return_dict=True)
#moving the model to GPU
model.to(dev)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
batch_size=4
num_of_batches=math.ceil(len(df_train)/batch_size)
num_of_batches

271

In [ ]:
# Initiating the Adafactor optimizer with recommended T5 settings
optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

In [ ]:
# Html based progress bar

from IPython.display import HTML, display
def progress(loss,value, max=100):
    return HTML(""" Batch loss :{loss}
      <progress    
    value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [ ]:
#Sets the module in training mode

num_of_epochs = 15

model.train()

loss_per_10_steps = []
for epoch in range(1,num_of_epochs+1):
    print('Running epoch: {}'.format(epoch))
  
    running_loss=0

    out = display(progress(1, num_of_batches+1), display_id=True)
    for i in range(num_of_batches):
        inputbatch = []
        labelbatch = []
        new_df = df_train[i*batch_size:i*batch_size+batch_size]
        for indx,row in new_df.iterrows():
            inputs = row['input_text']+'</s>' 
            labels = row['target_text']+'</s>'   
            inputbatch.append(inputs)
            labelbatch.append(labels)
        inputbatch = tokenizer.batch_encode_plus(inputbatch,padding=True,truncation=True,return_tensors='pt')["input_ids"]
        labelbatch = tokenizer.batch_encode_plus(labelbatch,padding=True,truncation=True,return_tensors="pt")["input_ids"]
        inputbatch = inputbatch.to(dev)
        labelbatch = labelbatch.to(dev)

        # clear out the gradients of all Variables 
        optimizer.zero_grad()

        # Forward propogation
        outputs = model(input_ids=inputbatch, labels=labelbatch)
        loss = outputs.loss
        loss_num=loss.item()
        logits = outputs.logits
        running_loss+=loss_num
        if i%10 ==0:      
            loss_per_10_steps.append(loss_num)
        out.update(progress(loss_num,i, num_of_batches+1))

        # calculating the gradients
        loss.backward()

        #updating the params
        optimizer.step()

    running_loss=running_loss/int(num_of_batches)
    print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))


Running epoch: 1


/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Epoch: 1 , Running loss: 1.0054126112443496
Running epoch: 2


Epoch: 2 , Running loss: 0.9616369956973734
Running epoch: 3


Epoch: 3 , Running loss: 0.9205098451283585
Running epoch: 4


...

Epoch: 11 , Running loss: 0.6477029707928865
Running epoch: 12


Epoch: 12 , Running loss: 0.6241181062816253
Running epoch: 13


Epoch: 13 , Running loss: 0.5957367826541852
Running epoch: 14


Epoch: 14 , Running loss: 0.5730258386953291
Running epoch: 15


Epoch: 15 , Running loss: 0.5488903985472183


In [ ]:
# torch.save(model.state_dict(),'pytorch_model.bin')

In [ ]:
df_test.head()

,input_text,target_text
0,"table_5 Link prediction results on the test-I,...",Results. Table 5 show the results in the filte...
1,table_3 Comparison on validation perplexity. B...,"In Figure 3, we present the validation perplex..."
2,table_4 Performance on maximally covered datas...,5.1 Maximum coverage comparison. Table 4 shows...
3,table_4 Open-ended and multiple-choice (MC) re...,4.4 Comparison to State-of-the-Art. Table 4 co...
4,table_1 Corpus size (length in token) and syst...,Results. From the whole corpus of 300 sentence...


In [ ]:
input_text = df_test['input_text'][18]
input_text

'table_5 Performance on monolingual word similarity computation with seed lexicon size 6000. Method BiLex Method CLSP-WR Method CLSP-SE Chinese (source) WS-240 Chinese (source) WS-297 English (target) WS-353 English (target) SL-999 accuracy accuracy accuracy accuracy 60.36 62.17 60.46 27.22 61.27 65.25 60.46 27.22 60.84 65.62 62.47 28.79'

In [ ]:

model.to(dev)

input_ids = tokenizer.encode(input_text, truncation=True, return_tensors='pt')
input_ids = input_ids.to(dev)
output = model.generate(input_ids=input_ids, max_length=512)
tokenizer.decode(output[0])

'<pad> Table 5 shows the performance of a CLSP-SE model on a monolingual word similarity. We can see that CLSP-WR performs well on a language model with Japanese (source) and Chinese (source) on the Chinese dataset (see the SL-999 dataset). It is also noticeable that on SL-999 (English) and SL-999 ( English), CLSP-SE performs better than T-Ls (WS-297) in SL-999 (WS-297 ( English).</s>'

## Playground

In [ ]:
import transformers
from datasets import load_dataset, load_metric

In [ ]:
dataset = load_dataset("yelp_review_full")

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})